In [2]:
import requests
import pendulum
from pathlib import Path
import os
import sys
from tsm import download
import concurrent.futures

In [3]:
diretorio_saida = Path(f'cfs_clima_91_20_daily/')
diretorio_saida.mkdir(exist_ok=True, parents=True)

In [ ]:
####################
# testando arquivos
####################

In [11]:
import xarray as xr

In [10]:
url1=f'https://ftp.emc.ncep.noaa.gov/cmb/sst/oisst_v2/GRIB/oisst.19811101.grb'
nome_arquivo1 = 'oisst.19811101.grb'

url2=f'https://www.ncei.noaa.gov/data/sea-surface-temperature-optimum-interpolation/v2.1/access/avhrr/198111/oisst-avhrr-v02r01.19811101.nc'
nome_arquivo2 = 'oisst-avhrr-v02r01.19811101.nc'

for url, nome_arquivo in zip([url1,url2], [nome_arquivo1, nome_arquivo2]):
    resp=requests.get(url)

    if resp.status_code == 200:
        resp=requests.get(url).content
        diretorio = Path(diretorio_saida, nome_arquivo)

        with open(diretorio, "wb") as arquivo_:
            arquivo_.write(resp)
            print(f"{nome_arquivo} [ok]")
            mensagem_de_sucesso = True

oisst.19811101.grb [ok]
oisst-avhrr-v02r01.19811101.nc [ok]


In [15]:
ds1 = xr.open_dataset(f'cfs_clima_91_20_daily/{nome_arquivo1}') 
ds2 = xr.open_dataset(f'cfs_clima_91_20_daily/{nome_arquivo2}') 

In [16]:
ds1

<xarray.Dataset>
Dimensions:     (latitude: 180, longitude: 360)
Coordinates:
    time        datetime64[ns] ...
    step        timedelta64[ns] ...
    surface     float64 ...
  * latitude    (latitude) float64 89.5 88.5 87.5 86.5 ... -87.5 -88.5 -89.5
  * longitude   (longitude) float64 -0.5 0.5 1.5 2.5 ... 355.5 356.5 357.5 358.5
    valid_time  datetime64[ns] ...
Data variables:
    t           (latitude, longitude) float32 ...
    atmw        (latitude, longitude) float32 ...
    p3091       (latitude, longitude) float32 ...
Attributes:
    GRIB_edition:            1
    GRIB_centre:             kwbc
    GRIB_centreDescription:  US National Weather Service - NCEP
    GRIB_subCentre:          4
    Conventions:             CF-1.7
    institution:             US National Weather Service - NCEP
    history:                 2022-09-28T16:52 GRIB to CDM+CF via cfgrib-0.9.1...

In [22]:
ds2

<xarray.Dataset>
Dimensions:  (time: 1, zlev: 1, lat: 720, lon: 1440)
Coordinates:
  * lat      (lat) float32 -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
  * lon      (lon) float32 0.125 0.375 0.625 0.875 ... 359.1 359.4 359.6 359.9
  * time     (time) datetime64[ns] 1981-11-01T12:00:00
  * zlev     (zlev) float32 0.0
Data variables:
    anom     (time, zlev, lat, lon) float32 ...
    err      (time, zlev, lat, lon) float32 ...
    ice      (time, zlev, lat, lon) float32 ...
    sst      (time, zlev, lat, lon) float32 ...
Attributes: (12/37)
    title:                      NOAA/NCEI 1/4 Degree Daily Optimum Interpolat...
    source:                     ICOADS, NCEP_GTS, GSFC_ICE, NCEP_ICE, Pathfin...
    id:                         oisst-avhrr-v02r01.19811101.nc
    naming_authority:           gov.noaa.ncei
    summary:                    NOAAs 1/4-degree Daily Optimum Interpolation ...
    cdm_data_type:              Grid
    ...                         ...
    metadata_link:              https://doi.org/10.25921/RE9P-PT57
    ncei_template_version:      NCEI_NetCDF_Grid_Template_v2.0
    comment:                    Data was converted from NetCDF-3 to NetCDF-4 ...
    sensor:                     Thermometer, AVHRR
    Conventions:                CF-1.6, ACDD-1.3
    references:                 Reynolds, et al.(2007) Daily High-Resolution-...

In [ ]:
##################
# fim teste
#################

In [4]:
lista = list()

inicio = pendulum.datetime(1991,1,1)
fim = pendulum.datetime(2020, 12, 31)
periodo = pendulum.period(inicio, fim)

for dt in periodo.range('days'):
    data_string = dt.format('DD-MM-YYYY')
    lista.append(data_string)
    #download(data_string=data_string)

In [ ]:
#######################################################################################################
# ATENÇÃO : ESTE BLOCO EXECUTARÁ O DOWNLOAD DE UMA GRANDE QUANTIDADE DE ARQUIVOS QUE JÁ FOI REALIZADA #
# RODÁ-LO DE NOVO NÃO VAI EXPLODIR O PC, MAS VAI GASTAR 10 MINUTOS DA SUA VIDA PARA NADA              #
#######################################################################################################

print(f"inicio: {pendulum.now().format('HH:mm:ss')}")

with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
        requisicoes = executor.map(download, lista)

print(f"fim: {pendulum.now().format('HH:mm:ss')}")

In [5]:
import xarray as xr

In [6]:
dataset = xr.open_mfdataset(f'{diretorio_saida}/*.nc', concat_dim="time", combine="nested")
                #data_vars='minimal', coords='minimal', compat='override',parallel=True)

In [7]:
dataset = dataset.isel(zlev=0)
dataset

<xarray.Dataset>
Dimensions:  (time: 10958, lat: 720, lon: 1440)
Coordinates:
  * lat      (lat) float32 -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
  * lon      (lon) float32 0.125 0.375 0.625 0.875 ... 359.1 359.4 359.6 359.9
  * time     (time) datetime64[ns] 1991-01-01T12:00:00 ... 2020-12-31T12:00:00
    zlev     float32 0.0
Data variables:
    anom     (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    err      (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    ice      (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    sst      (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
Attributes: (12/37)
    title:                      NOAA/NCEI 1/4 Degree Daily Optimum Interpolat...
    source:                     ICOADS, NCEP_GTS, GSFC_ICE, NCEP_ICE, Pathfin...
    id:                         oisst-avhrr-v02r01.19910101.nc
    naming_authority:           gov.noaa.ncei
    summary:                    NOAAs 1/4-degree Daily Optimum Interpolation ...
    cdm_data_type:              Grid
    ...                         ...
    metadata_link:              https://doi.org/10.25921/RE9P-PT57
    ncei_template_version:      NCEI_NetCDF_Grid_Template_v2.0
    comment:                    Data was converted from NetCDF-3 to NetCDF-4 ...
    sensor:                     Thermometer, AVHRR
    Conventions:                CF-1.6, ACDD-1.3
    references:                 Reynolds, et al.(2007) Daily High-Resolution-...

# geração de climatologia

In [9]:
# para criar climatologia
anom_tsm = dataset.sst.groupby('time.month').mean()

In [10]:
import pandas as pd
from regioes_indices import regioes2
import xarray as xr
import pendulum

import recorta_dado
from utils import padronizar_longitude
from utils import delimitar_regiao
from utils import converter_para_celsius

In [11]:
def delimitar_regiao2(obj: xr.DataArray, indice) -> xr.DataArray:
    """
    POR ALGUM MOTIVO QUE NÃO SEI EXPLICAR FOI PRECISO INVERTER A ORDEM DA LATITUDE PARA O RECORTE FUNCIONAR.
    
    Delimitação de regiões do oceano Atlântico e seu valor médio na região.
    Para um dado índice.
    Parameters
    ----------
    obj : xr.DataArray
        Dados da rodada do modelo
    indice : Any
        Informações sobre o índice vindos do módulo schema.nino
    Returns
    -------
    xr.DataArray
        Série com as temperaturas previstas.
    """
    rec = obj.sel(
        lat=slice(indice.latitude[1], indice.latitude[0]),
        lon=slice(indice.longitude[0], indice.longitude[1]),
    ).mean(dim=["lat", "lon"])
 
    valor = rec.compute()

    return valor

In [12]:
regioes2

[Indices(id=12, latitude=(0, -10), longitude=(270, 280), nome='TSM Niño 1+2', longitude_padronizada=False),
 Indices(id=3, latitude=(5, -5), longitude=(210, 270), nome='TSM Niño 3', longitude_padronizada=False),
 Indices(id=34, latitude=(5, -5), longitude=(190, 240), nome='TSM Niño 3.4', longitude_padronizada=False),
 Indices(id=4, latitude=(5, -5), longitude=(200, 210), nome='TSM Niño 4', longitude_padronizada=False),
 Indices(id=5, latitude=(-33, -43), longitude=(-63, -48), nome='TSM CBM', longitude_padronizada=True),
 Indices(id=6, latitude=(25, 5), longitude=(-55, -15), nome='TSM TNA', longitude_padronizada=True),
 Indices(id=7, latitude=(0, -20), longitude=(-30, 10), nome='TSM TSA', longitude_padronizada=True)]

In [13]:
dicionario = {
    'mes': [],
    "TSM Niño 1+2" : [],
    "TSM Niño 3" : [],
    "TSM Niño 3.4" : [],
    "TSM Niño 4" : [],
    "TSM CBM" : [],
    "TSM TNA" : [],
    "TSM TSA" : [],
}

for mes in range(1,12+1):
    tsm = anom_tsm.sel(month=mes)
    dicionario['mes'].append(mes)
    
    for indice in regioes2:
        tsm_ = tsm.compute()
        
        if indice.longitude_padronizada:
            tsm__ = padronizar_longitude(tsm_, xdim='lon')
            valor = delimitar_regiao2(obj=tsm__, indice=indice)
        else:
            tsm__ = tsm_
            valor = delimitar_regiao2(obj=tsm__, indice=indice)
        
        valor = round(float(valor.values), 2)

        dicionario[indice.nome].append(valor)

        print(f'O valor da região {indice.nome} no mês {mes} é : {valor}')

O valor da região TSM Niño 1+2 no mês 1 é : 24.5
O valor da região TSM Niño 3 no mês 1 é : 25.6
O valor da região TSM Niño 3.4 no mês 1 é : 26.49
O valor da região TSM Niño 4 no mês 1 é : 26.96
O valor da região TSM CBM no mês 1 é : 19.41
O valor da região TSM TNA no mês 1 é : 24.87
O valor da região TSM TSA no mês 1 é : 25.58
O valor da região TSM Niño 1+2 no mês 2 é : 26.05
O valor da região TSM Niño 3 no mês 2 é : 26.32
O valor da região TSM Niño 3.4 no mês 2 é : 26.67
O valor da região TSM Niño 4 no mês 2 é : 26.98
O valor da região TSM CBM no mês 2 é : 20.04
O valor da região TSM TNA no mês 2 é : 24.3
O valor da região TSM TSA no mês 2 é : 26.48
O valor da região TSM Niño 1+2 no mês 3 é : 26.49
O valor da região TSM Niño 3 no mês 3 é : 27.12
O valor da região TSM Niño 3.4 no mês 3 é : 27.18
O valor da região TSM Niño 4 no mês 3 é : 27.32
O valor da região TSM CBM no mês 3 é : 19.58
O valor da região TSM TNA no mês 3 é : 24.25
O valor da região TSM TSA no mês 3 é : 27.03
O valor da

In [14]:
dt = pd.DataFrame.from_dict(dicionario)

In [15]:
dt.set_index('mes').to_csv('oisstv2_clima_1991-2020.csv', sep=';')

# Geração de histórico

In [16]:
import pandas as pd
from pathlib import Path
import pendulum
import xarray as xr

from tsm import download
from regioes_indices import regioes
from utils import padronizar_longitude
import concurrent.futures

In [17]:
def delimitar_regiao2(obj: xr.DataArray, indice) -> xr.DataArray:
    """
    POR ALGUM MOTIVO QUE NÃO SEI EXPLICAR FOI PRECISO INVERTER A ORDEM DA LATITUDE PARA O RECORTE FUNCIONAR.
    
    Delimitação de regiões do oceano Atlântico e seu valor médio na região.
    Para um dado índice.
    Parameters
    ----------
    obj : xr.DataArray
        Dados da rodada do modelo
    indice : Any
        Informações sobre o índice vindos do módulo schema.nino
    Returns
    -------
    xr.DataArray
        Série com as temperaturas previstas.
    """
    rec = obj.sel(
        lat=slice(indice.latitude[1], indice.latitude[0]),
        lon=slice(indice.longitude[0], indice.longitude[1]),
    ).mean(dim=["lat", "lon"])
 
    valor = rec.compute()

    return valor

In [18]:
def to_month_of_year(ds):
    
    year = ds.time.dt.year
    month = ds.time.dt.month

    # assign new coords
    ds = ds.assign_coords(year=("time", year.data), month=("time", month.data))

    # reshape the array to (..., "month", "year")
    ds_ = ds.set_index(time=("year", "month")).unstack("time")
    
    return ds_

In [19]:
diretorio_saida = Path(f'historico_oiv2/')
diretorio_saida.mkdir(exist_ok=True, parents=True)

In [20]:
lista_datas = list()
lista_com_falses = list()
lista_com_diretorio_saida = list()

inicio = pendulum.datetime(2020,1,1)
fim = pendulum.now()
periodo = pendulum.period(inicio, fim)

for dt in periodo.range('days'):
    data_string = dt.format('DD-MM-YYYY')
    
    lista_datas.append(data_string)
    lista_com_falses.append(False)
    lista_com_diretorio_saida.append(diretorio_saida)

In [ ]:
print(f"inicio: {pendulum.now().format('HH:mm:ss')}")

with concurrent.futures.ThreadPoolExecutor(max_workers=100) as executor:
        requisicoes = executor.map(download, lista_datas, lista_com_falses, lista_com_diretorio_saida)

print(f"fim: {pendulum.now().format('HH:mm:ss')}")

In [21]:
diretorio_saida

PosixPath('historico_oiv2')

In [22]:
dataset = xr.open_mfdataset(f'{diretorio_saida}/*.nc', concat_dim="time", combine="nested",
                data_vars='minimal', coords='minimal', compat='override',parallel=True)

dataset = dataset.isel(zlev=0)

In [23]:
dataset_ = dataset.resample(time="M").mean()

In [24]:
ds = to_month_of_year(dataset_)

In [25]:
dicionario2 = {
    'data'   : [],
    "TSM Niño 1+2" : [],
    "TSM Niño 1+2 anom" : [],
    "TSM Niño 3" : [],
    "TSM Niño 3 anom" : [],
    "TSM Niño 3.4" : [],
    "TSM Niño 3.4 anom" : [],
    "TSM Niño 4" : [],
    "TSM Niño 4 anom" : [],
    "TSM CBM" : [],
    "TSM CBM anom" : [],
    "TSM TNA" : [],
    "TSM TNA anom" : [],
    "TSM TSA" : [],
    "TSM TSA anom" : [],
}

for dt in periodo.range('months'):
    
    ano = int(dt.format('YYYY'))
    mes = int(dt.format('MM'))
    
    ds_ = ds.sel(year=ano, month=mes)
    dicionario2['data'].append(dt.format('YYYY-MM-DD'))
    
    for indice in regioes2:
        
        if indice.longitude_padronizada:
            tsm = padronizar_longitude(ds_.sst, xdim='lon')
            valor_tsm = delimitar_regiao2(obj=tsm, indice=indice)
            valor_tsm = round(float(valor_tsm.values), 2)


            anom = padronizar_longitude(ds_.anom, xdim='lon')
            valor_anom = delimitar_regiao2(obj=anom, indice=indice)
            valor_anom = round(float(valor_anom.values), 2)
            
        else:
            tsm = ds_.sst
            valor_tsm = delimitar_regiao2(obj=tsm, indice=indice)
            valor_tsm = round(float(valor_tsm.values), 2)


            anom = ds_.anom
            valor_anom = delimitar_regiao2(obj=anom, indice=indice)
            valor_anom = round(float(valor_anom.values), 2)

            
        dicionario2[indice.nome].append(valor_tsm)
        dicionario2[f"{indice.nome} anom"].append(valor_anom)
        
        print(f'O valor da região {indice.nome} em {dt.format("YYYY-MM-DD")} é : {valor_tsm} e {valor_anom}')

O valor da região TSM Niño 1+2 em 2020-01-01 é : 24.51 e 0.05
O valor da região TSM Niño 3 em 2020-01-01 é : 25.8 e 0.15
O valor da região TSM Niño 3.4 em 2020-01-01 é : 27.02 e 0.48
O valor da região TSM Niño 4 em 2020-01-01 é : 27.66 e 0.67
O valor da região TSM CBM em 2020-01-01 é : 19.02 e -0.02
O valor da região TSM TNA em 2020-01-01 é : 24.87 e 0.15
O valor da região TSM TSA em 2020-01-01 é : 26.68 e 1.19
O valor da região TSM Niño 1+2 em 2020-02-01 é : 26.25 e 0.41
O valor da região TSM Niño 3 em 2020-02-01 é : 26.4 e 0.04
O valor da região TSM Niño 3.4 em 2020-02-01 é : 27.02 e 0.28
O valor da região TSM Niño 4 em 2020-02-01 é : 27.55 e 0.57
O valor da região TSM CBM em 2020-02-01 é : 19.72 e 0.04
O valor da região TSM TNA em 2020-02-01 é : 24.65 e 0.42
O valor da região TSM TSA em 2020-02-01 é : 27.23 e 0.87
O valor da região TSM Niño 1+2 em 2020-03-01 é : 26.94 e 0.73
O valor da região TSM Niño 3 em 2020-03-01 é : 27.29 e 0.25
O valor da região TSM Niño 3.4 em 2020-03-01 é : 

In [26]:
dt2 = pd.DataFrame.from_dict(dicionario2)

In [27]:
dt2.set_index('data').to_csv('oiv2_HistoricoNovo_2020-2022.csv', sep=';')